Чтобы оптимизировать производственные расходы, металлургический комбинат ООО «Так закаляем сталь» решил уменьшить потребление электроэнергии на этапе обработки стали. Нам предстоит построить модель, которая будет предсказывать температуру стали конечную температуру после всех присыпок.

### План:
- загрузить данные и привести к нужным форматам
- провести исследовательский анализ
    - выявить особенности и различные факторы для обучения модели
    - выявить аномалии
    - проверить на адекватность
- Выводим таргет и факторы по которым будет обучаться и предсказывать модель
    - ТАРГЕТ: Вывести последнюю температуру по партиям и удалить партии, где после последнего замера были еще присыпки
    - Первая температура по партии
    - Активную температуру по партии
    - Длительность нагрева
    - Суммировать данные по сыпучим матерьялам по партиям
    - Суммировать данные по газам по партиям
    - Суммировать данные по проволочным матерьялам по партиям
    - Первая температура по партии
- Делим получившийся датафрейм на обучающую, валидационную и тестовую выборки
- Пробуем разные модели и подгоняем гиперпараметры. Оперделяем лучшую модель.
- Тестируем лучшую модель на тестовой выборке
- Пишем выводы

### Описание данных

Данные состоят из файлов, полученных из разных источников:

- `data_arc.csv` — данные об электродах;
- `data_bulk.csv` — данные о подаче сыпучих материалов (объём);
- `data_bulk_time.csv` *—* данные о подаче сыпучих материалов (время);
- `data_gas.csv` — данные о продувке сплава газом;
- `data_temp.csv` — результаты измерения температуры;
- `data_wire.csv` — данные о проволочных материалах (объём);
- `data_wire_time.csv` — данные о проволочных материалах (время).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np     
import datetime as dt

Открываем датафреймы, сразу приводя индексы и даты к порядку

In [2]:
# file = '/Users/churk/1.Проекты/Выпускной/final_steel/'
file = '/datasets/final_steel/'
data_arc = pd.read_csv(file + r"data_arc.csv", index_col=0, parse_dates=['Начало нагрева дугой', 'Конец нагрева дугой'])
data_bulk = pd.read_csv(file + r"data_bulk.csv", index_col=0)
data_bulk_time = pd.read_csv(file + r"data_bulk_time.csv", index_col=0)
data_bulk_time = pd.read_csv(file + r"data_bulk_time.csv", index_col=0, parse_dates=data_bulk_time.columns.tolist())
data_gas = pd.read_csv(file + r"data_gas.csv", index_col=0)
data_temp = pd.read_csv(file + r"data_temp.csv", index_col=0,  parse_dates=['Время замера'])
data_wire = pd.read_csv(file + r"data_wire.csv", index_col=0)
data_wire_time = pd.read_csv(file + r"data_wire_time.csv", index_col=0)
data_wire_time = pd.read_csv(file + r"data_wire_time.csv", index_col=0, parse_dates=data_wire_time.columns.tolist())
data_wire_time

,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
key,,,,,,,,,
1,2019-05-03 11:11:41,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,2019-05-03 11:46:10,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,2019-05-03 12:13:47,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,2019-05-03 12:48:05,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,2019-05-03 13:18:15,2019-05-03 13:32:06,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...
3237,2019-08-31 22:50:20,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3238,2019-08-31 23:38:24,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3239,2019-09-01 01:50:43,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [3]:
data_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15907 entries, 1 to 3241
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Время замера  15907 non-null  datetime64[ns]
 1   Температура   13006 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 372.8 KB


# Исследовательский анализ
- Сейчас мы начнем по кусочкам формировать датафрейм по которому будет обучаться модель. Ну и будем проверять каждый кусочек на различные анамалии и общую адекватность.
- таргетом нашей будущей модели являеться температура последнего замера, которые указаны в датафрейме data_temp
Далее идут признаки на основании которых модель будет принимать решения и вообще обучаться
- Первая температура по партии
- Длительность обработки
- Активную мощьность по партии
- Суммировать данные по сыпучим матерьялам по партиям
- Суммировать данные по газам по партиям
- Суммировать данные по проволочным матерьялам по партиям

Находим последний замер и добавляем к нему температуру.

In [4]:
df = data_temp.pivot_table(index=['key'], values=['Время замера', 'Температура'], aggfunc='max')
df

,Время замера,Температура
key,,
1,2019-05-03 11:30:39,1618.0
2,2019-05-03 11:59:12,1608.0
3,2019-05-03 12:34:57,1599.0
4,2019-05-03 12:59:25,1625.0
5,2019-05-03 13:36:01,1602.0
...,...,...
3237,2019-08-31 23:05:43,1569.0
3238,2019-08-31 23:43:00,1584.0
3239,2019-09-01 02:24:15,1598.0


Проверяем на аномалии и адекватность

In [5]:
df.describe()

,Температура
count,3216.000000
mean,1600.044465
std,22.205940
min,1204.000000
25%,1589.000000
50%,1598.000000
75%,1611.000000
max,1705.000000


Разброс по температурам достаточно большой, но данные распределенны равномерно, так что аномалий нет.
Теперь найдем те замеры, после которых еще были присыпки

In [6]:
data_bulk_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3129 entries, 1 to 3241
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Bulk 1   252 non-null    datetime64[ns]
 1   Bulk 2   22 non-null     datetime64[ns]
 2   Bulk 3   1298 non-null   datetime64[ns]
 3   Bulk 4   1014 non-null   datetime64[ns]
 4   Bulk 5   77 non-null     datetime64[ns]
 5   Bulk 6   576 non-null    datetime64[ns]
 6   Bulk 7   25 non-null     datetime64[ns]
 7   Bulk 8   1 non-null      datetime64[ns]
 8   Bulk 9   19 non-null     datetime64[ns]
 9   Bulk 10  176 non-null    datetime64[ns]
 10  Bulk 11  177 non-null    datetime64[ns]
 11  Bulk 12  2450 non-null   datetime64[ns]
 12  Bulk 13  18 non-null     datetime64[ns]
 13  Bulk 14  2806 non-null   datetime64[ns]
 14  Bulk 15  2248 non-null   datetime64[ns]
dtypes: datetime64[ns](15)
memory usage: 391.1 KB


У нас достаточно много пропусков в последнем столбце, так что чтобы узнать время последней присыпки придется их заполнить. Метод apply в данном случае на 3000 строк весьма шустро справиться)

In [7]:
def final_bulk_past(stroka):
    stroka = stroka.dropna()
    return stroka[-1]

data_bulk_time['final_bulk'] = data_bulk_time.apply(final_bulk_past, axis=1)  
data_bulk_time.head()

,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15,final_bulk
key,,,,,,,,,,,,,,,,
1,NaT,NaT,NaT,2019-05-03 11:21:30,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 11:03:52,NaT,2019-05-03 11:03:52,2019-05-03 11:03:52,2019-05-03 11:03:52
2,NaT,NaT,NaT,2019-05-03 11:46:38,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 11:40:20,NaT,2019-05-03 11:40:20,2019-05-03 11:40:20,2019-05-03 11:40:20
3,NaT,NaT,NaT,2019-05-03 12:31:06,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 12:09:40,NaT,2019-05-03 12:09:40,2019-05-03 12:09:40,2019-05-03 12:09:40
4,NaT,NaT,NaT,2019-05-03 12:48:43,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 12:41:24,NaT,2019-05-03 12:41:24,2019-05-03 12:41:24,2019-05-03 12:41:24
5,NaT,NaT,NaT,2019-05-03 13:18:50,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-03 13:12:56,NaT,2019-05-03 13:12:56,2019-05-03 13:12:56,2019-05-03 13:12:56


проверяем как заполненны пропуски

In [8]:
data_bulk_time[data_bulk_time['Bulk 15'].isnull()].head()

,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15,final_bulk
key,,,,,,,,,,,,,,,,
19,NaT,NaT,NaT,NaT,NaT,2019-05-04 00:53:02,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-04 00:53:02
20,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-04 01:44:28,NaT,2019-05-04 01:44:28
22,NaT,NaT,2019-05-04 04:29:57,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-04 04:24:59,NaT,2019-05-04 04:24:59
26,NaT,NaT,2019-05-04 07:28:33,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-04 07:28:33
28,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-05-04 08:30:00,NaT,2019-05-04 08:30:00,NaT,2019-05-04 08:30:00


In [9]:
data_bulk_time[data_bulk_time['final_bulk'].isnull()]

,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15,final_bulk
key,,,,,,,,,,,,,,,,


Пропуски заполненны как задуманно по последней присыпке и полностью.  
Убираем из таргета те строки, которые произошли раньше последней присыпки

In [10]:
data_bulk_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3129 entries, 1 to 3241
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Bulk 1      252 non-null    datetime64[ns]
 1   Bulk 2      22 non-null     datetime64[ns]
 2   Bulk 3      1298 non-null   datetime64[ns]
 3   Bulk 4      1014 non-null   datetime64[ns]
 4   Bulk 5      77 non-null     datetime64[ns]
 5   Bulk 6      576 non-null    datetime64[ns]
 6   Bulk 7      25 non-null     datetime64[ns]
 7   Bulk 8      1 non-null      datetime64[ns]
 8   Bulk 9      19 non-null     datetime64[ns]
 9   Bulk 10     176 non-null    datetime64[ns]
 10  Bulk 11     177 non-null    datetime64[ns]
 11  Bulk 12     2450 non-null   datetime64[ns]
 12  Bulk 13     18 non-null     datetime64[ns]
 13  Bulk 14     2806 non-null   datetime64[ns]
 14  Bulk 15     2248 non-null   datetime64[ns]
 15  final_bulk  3129 non-null   datetime64[ns]
dtypes: datetime64[ns](16)
me

In [11]:
df = df[((df['Время замера'] - data_bulk_time['final_bulk']).dt.days >= 0)]
df

,Время замера,Температура
key,,
1,2019-05-03 11:30:39,1618.0
2,2019-05-03 11:59:12,1608.0
3,2019-05-03 12:34:57,1599.0
4,2019-05-03 12:59:25,1625.0
5,2019-05-03 13:36:01,1602.0
...,...,...
3237,2019-08-31 23:05:43,1569.0
3238,2019-08-31 23:43:00,1584.0
3239,2019-09-01 02:24:15,1598.0


избавились примерно от ста строк с досыпками и пропусками

### Первая температура по партии и общее время обработки

In [12]:
g = data_temp.pivot_table(index=['key'], values=['Время замера', 'Температура'], aggfunc='min')
g.columns = 'первый_замер', 'входная_температура'
df = df.merge(g,  how='inner', left_index = True, right_index = True)
df['время_обработки'] = df['Время замера'] - df['первый_замер']


In [13]:
df.describe()

,Температура,входная_температура,время_обработки
count,3125.000000,3125.00000,3125
mean,1599.585280,1580.20640,0 days 00:31:01.695680
std,21.616733,25.05934,0 days 00:20:53.949096246
min,1204.000000,1191.00000,0 days 00:00:00
25%,1589.000000,1569.00000,0 days 00:19:52
50%,1598.000000,1581.00000,0 days 00:26:42
75%,1611.000000,1591.00000,0 days 00:37:39
max,1702.000000,1684.00000,0 days 06:32:17


- у нас есть две аномалии: с нулевым временем обработки и 6 часовой. Нулевое время только одно. Поэтому его спокойно удалим
- время между первым и последним замером измерятся в минутах, поэтому для удобства модели мы пересохраним данные в минутах


In [14]:
df = df[df['время_обработки'] != '0 days 00:00:00']

In [15]:
df[df['время_обработки'] > '01:00:00']

,Время замера,Температура,первый_замер,входная_температура,время_обработки
key,,,,,
44,2019-05-05 01:13:17,1619.0,2019-05-04 22:13:44,1570.0,0 days 02:59:33
45,2019-05-05 00:26:13,1629.0,2019-05-04 23:06:36,1538.0,0 days 01:19:37
46,2019-05-05 03:14:53,1617.0,2019-05-04 23:54:43,1538.0,0 days 03:20:10
115,2019-05-07 12:39:22,1638.0,2019-05-07 11:01:24,1547.0,0 days 01:37:58
118,2019-05-07 18:53:53,1652.0,2019-05-07 16:13:55,1566.0,0 days 02:39:58
...,...,...,...,...,...
2964,2019-08-23 02:26:53,1534.0,2019-08-23 01:02:05,1534.0,0 days 01:24:48
2965,2019-08-23 02:57:45,1577.0,2019-08-23 00:35:15,1577.0,0 days 02:22:30
3119,2019-08-28 06:04:21,1568.0,2019-08-28 03:44:38,1568.0,0 days 02:19:43


Итого: обработка больше часа это большая редкость - 128 выбросов. Уберем подобные выбросы

In [16]:
df = df[df['время_обработки'] < '01:00:00']

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2890 entries, 1 to 3241
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   Время замера         2890 non-null   datetime64[ns] 
 1   Температура          2890 non-null   float64        
 2   первый_замер         2890 non-null   datetime64[ns] 
 3   входная_температура  2890 non-null   float64        
 4   время_обработки      2890 non-null   timedelta64[ns]
 5   Активная мощность    2890 non-null   float64        
 6   присыпки             2890 non-null   float64        
 7   Газ 1                2890 non-null   float64        
 8   прочие_матерьялы     2890 non-null   float64        
dtypes: datetime64[ns](2), float64(6), timedelta64[ns](1)
memory usage: 225.8 KB


переводим время обработки в минуты

In [25]:
df['время_обработки'] = df['время_обработки'].dt.seconds/60

### Прибавляем прочие агрегированные столбцы
- Суммируем активную мощьность и мерджим с нашим датафреймом
- Суммируем данные по сыпучим матерьялам по партиям
- Суммируем данные по газам по партиям
- Суммируем данные по проволочным матерьялам по партиям
- Проверяем на выбросы и артефакты

In [17]:
data_arc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14876 entries, 1 to 3241
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Начало нагрева дугой  14876 non-null  datetime64[ns]
 1   Конец нагрева дугой   14876 non-null  datetime64[ns]
 2   Активная мощность     14876 non-null  float64       
 3   Реактивная мощность   14876 non-null  float64       
dtypes: datetime64[ns](2), float64(2)
memory usage: 581.1 KB


In [18]:
df = df.merge(data_arc.pivot_table(index=['key'], values=['Активная мощность'], aggfunc='sum'), 
              how='inner', left_index = True, right_index = True)
df = df.merge(pd.DataFrame(data={'присыпки':data_bulk.sum(axis=1)}), how='inner', left_index = True, right_index = True)
df = df.merge(data_gas, how='inner', left_index = True, right_index = True)
df = df.merge(pd.DataFrame(data={'прочие_матерьялы':data_wire.sum(axis=1)}), how='inner', left_index = True, right_index = True)
df

,Время замера,Температура,первый_замер,входная_температура,время_обработки,Активная мощность,присыпки,Газ 1,прочие_матерьялы
key,,,,,,,,,
1,2019-05-03 11:30:39,1618.0,2019-05-03 11:16:18,1571.0,0 days 00:14:21,4.878147,553.0,29.749986,60.059998
2,2019-05-03 11:59:12,1608.0,2019-05-03 11:37:27,1577.0,0 days 00:21:45,3.052598,582.0,12.555561,96.052315
3,2019-05-03 12:34:57,1599.0,2019-05-03 12:13:17,1596.0,0 days 00:21:40,2.525882,544.0,28.554793,91.160157
4,2019-05-03 12:59:25,1625.0,2019-05-03 12:52:57,1601.0,0 days 00:06:28,3.209250,595.0,18.841219,89.063515
5,2019-05-03 13:36:01,1602.0,2019-05-03 13:23:19,1576.0,0 days 00:12:42,3.347173,584.0,5.413692,98.352796
...,...,...,...,...,...,...,...,...,...
3237,2019-08-31 23:05:43,1569.0,2019-08-31 22:44:04,1569.0,0 days 00:21:39,3.932467,758.0,5.543905,38.088959
3238,2019-08-31 23:43:00,1584.0,2019-08-31 23:30:31,1584.0,0 days 00:12:29,2.106529,594.0,6.745669,56.128799
3239,2019-09-01 02:24:15,1598.0,2019-09-01 01:31:47,1598.0,0 days 00:52:28,4.666754,630.0,16.023518,143.357761


In [26]:
df.describe()

,Температура,входная_температура,время_обработки,Активная мощность,присыпки,Газ 1,прочие_матерьялы
count,2890.000000,2890.000000,2890.000000,2890.000000,2890.000000,2890.000000,2890.000000
mean,1597.507266,1578.689619,28.533189,3.019566,590.179585,10.926766,122.644443
std,19.544823,23.712120,11.412616,1.244534,266.467553,5.721006,65.241615
min,1204.000000,1191.000000,3.283333,0.186226,19.000000,0.008399,1.918800
25%,1588.000000,1568.000000,19.866667,2.116967,432.500000,7.189030,86.065194
50%,1597.000000,1580.000000,26.350000,2.956942,588.000000,9.856927,113.292917
75%,1608.000000,1590.000000,36.145833,3.810941,762.000000,13.668087,148.274104
max,1662.000000,1651.000000,59.783333,9.129200,1569.000000,60.935689,663.851766


In [32]:
df[df['прочие_матерьялы'] > 500]

,Время замера,Температура,первый_замер,входная_температура,время_обработки,Активная мощность,присыпки,Газ 1,прочие_матерьялы
key,,,,,,,,,
407,2019-05-18 13:49:12,1589.0,2019-05-18 13:22:12,1554.0,27.000000,4.007215,359.0,16.194449,517.560783
1840,2019-07-07 19:35:24,1602.0,2019-07-07 18:55:58,1579.0,39.433333,2.268302,122.0,13.767550,568.777664
1841,2019-07-07 20:19:54,1589.0,2019-07-07 19:51:44,1559.0,28.166667,3.722151,169.0,13.814996,504.331137
1915,2019-07-15 20:51:27,1597.0,2019-07-15 20:03:15,1570.0,48.200000,4.417417,275.0,12.918119,568.237950
1916,2019-07-15 22:15:44,1614.0,2019-07-15 21:20:54,1562.0,54.833333,2.334628,231.0,18.867788,506.304676
2501,2019-08-06 04:31:14,1587.0,2019-08-06 03:44:42,1587.0,46.533333,4.164288,665.0,13.828957,625.669418
2502,2019-08-06 05:12:20,1583.0,2019-08-06 04:45:26,1583.0,26.900000,2.279659,178.0,13.890142,521.061852
2507,2019-08-06 09:29:03,1568.0,2019-08-06 09:01:38,1568.0,27.416667,3.499325,406.0,16.405663,525.713151
2508,2019-08-06 10:30:57,1586.0,2019-08-06 09:43:13,1586.0,47.733333,3.524302,416.0,18.313311,663.851766


In [34]:
df[df['Активная мощность'] > 8]

,Время замера,Температура,первый_замер,входная_температура,время_обработки,Активная мощность,присыпки,Газ 1,прочие_матерьялы
key,,,,,,,,,
813,2019-06-02 13:37:17,1603.0,2019-06-02 12:43:41,1548.0,53.60,8.070847,488.0,19.856740,114.391678
1051,2019-06-10 22:13:03,1637.0,2019-06-10 21:25:12,1554.0,47.85,9.129200,1120.0,16.447339,121.165196


## Выводы по исследовательскому анализу
- Есть выбивающиеся значения в активной мощьности, присыпках и прочих матерьялах, но их стоит оставить, так как они свойственны долгим обработкам

### Уточнения:
- Я не уверен на счет правильности своего решения по поводу исключения из выборки партий с обработкой больше часа. Хотя возможно лучше бы удалить выбросы и из колонок с мощьностью и присыпками, но не является ли читерством формирование таких подбных "уютных" выборок?

наброски не забыть:
- на этот блок. Определить графики, изучить. Первичный анализ данных
- использовать начальную температуру как фактор
- убрать перерасход тепла
- уменьшение циклов нагрев-остывание
- контролировать последнюю нагрев
- Цель спрогнозировать температуру ковша после, регрессивной модели.
- Параметры: объем материала, используем газ, затраченная мощьность по партии, начальная температура
- Суммировать мощьности.
- Реактивная мощьность - не использовать. Так как она вредная и не идет на нагрев.
- Анализ важности факторов. Ранжирование факторов влияющих на целевуюкак
Булктайм и вайттайм - нужны для определения леквидных данных
Партии в которых происходили еще какие то операции после финального замера
Метрики MAE - 6 процентов